<a href="https://colab.research.google.com/github/kikosikera/2020_04_04_pd_sk_fq_tb/blob/master/2019_10_18_pd_sk_fq_tb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Demographics: Neighborhood Similarities > New York and Toronto



Given a neighborhood (through Postcode) how to find the 3 most common venue categories in Toronto and New York City combined.
  
We can use the Foursquare location data to find neighborhoods with similar nearby venues in both cities. 

### Import Libraries and Datasets

In [0]:
# Download all the dependencies that we will need.
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from bs4 import BeautifulSoup # library to parse html from web page
import requests
from IPython.display import display_html

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Download and Explore Dataset

#### Load and explore the data of New York
- New York neighborhood has a total of 5 boroughs and 306 neighborhoods. 
- The dataset is on the following link https://geo.nyu.edu/catalog/nyu_2451_34572. 
- The dataset includes Borough, Latitude and Logitude of each Neighborhood.

In [0]:
!wget -q -O newyork_data.json https://geo.nyu.edu/download/file/nyu-2451-34572-geojson.json
print('Data downloaded!')

Data downloaded!


In [0]:
with open("newyork_data.json") as json_data:
    newyork_data = json.load(json_data)

newyork_neighborhoods_data = newyork_data['features']

# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
newyork_neighborhoods = pd.DataFrame(columns=column_names)

for data in newyork_neighborhoods_data:
    borough = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
    
    neighborhood_name = neighborhood_name + ', ' + borough
    
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    newyork_neighborhoods = newyork_neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
newyork_neighborhoods.head()  

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,"Wakefield, Bronx",40.894705,-73.847201
1,Bronx,"Co-op City, Bronx",40.874294,-73.829939
2,Bronx,"Eastchester, Bronx",40.887556,-73.827806
3,Bronx,"Fieldston, Bronx",40.895437,-73.905643
4,Bronx,"Riverdale, Bronx",40.890834,-73.912585


In [0]:
print('The New York dataframe has {} boroughs and {} neighborhoods.'.format(
        len(newyork_neighborhoods['Borough'].unique()),
        newyork_neighborhoods.shape[0]
    )
)

The New York dataframe has 5 boroughs and 306 neighborhoods.


In [0]:
newyork_neighborhoods.to_csv(r"/content/drive/My Drive/Colab Notebooks/Data/PROJ/2019_10_18_pd_sk_fq_tb/data/newyork_neighborhoods.csv", sep = ',')

In [0]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')
table = soup.find('table')
table_rows = pd.read_html(str(table))[0]

In [0]:
table_rows.to_csv(r"/content/drive/My Drive/Colab Notebooks/Data/PROJ/2019_10_18_pd_sk_fq_tb/data/table_rows.csv", sep = ',')

In [0]:
table_rows = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Data/PROJ/2019_10_18_pd_sk_fq_tb/data/table_rows.csv", sep = ',', low_memory=False)

In [0]:
table_rows.head()

,Unnamed: 0,Postal code,Borough,Neighborhood
0,0,M1A,Not assigned,NaN
1,1,M2A,Not assigned,NaN
2,2,M3A,North York,Parkwoods
3,3,M4A,North York,Victoria Village
4,4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [0]:
table_rows_etl = table_rows.copy()
table_rows_etl = table_rows_etl.drop('Unnamed: 0', 1)
table_rows_etl = table_rows_etl[~table_rows_etl['Borough'].isin(['Not assigned'])]

In [0]:
table_rows_etl.rename(index=str, columns={"Postal code": "PostalCode"}, inplace=True)

In [0]:
table_rows_etl.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [0]:
table_rows_etl.to_csv(r"/content/drive/My Drive/Colab Notebooks/Data/PROJ/2019_10_18_pd_sk_fq_tb/data/table_rows_etl.csv", sep = ',')

In [0]:
table_rows_etl = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Data/PROJ/2019_10_18_pd_sk_fq_tb/data/table_rows_etl.csv", sep = ',', low_memory=False)

Build dataframe for all neighborhoods in Canada

In [0]:
df = table_rows_etl.copy()

In [0]:
df.shape

(103, 4)

Add latitude and longitude to dataframe from https://cocl.us/Geospatial_data

In [0]:
geocode = pd.read_csv("https://cocl.us/Geospatial_data")
geocode.rename(index=str, columns={"Postal Code": "PostalCode"}, inplace=True)
df = df.merge(geocode, left_on='PostalCode', right_on='PostalCode')

In [0]:
geocode.to_csv(r"/content/drive/My Drive/Colab Notebooks/Data/PROJ/2019_10_18_pd_sk_fq_tb/data/geocode.csv", sep = ',')

Get neighborhoods for boroughs contain word 'Toronto'

In [0]:
toronto_neighborhoods = df[df['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_neighborhoods['Neighborhood'] = toronto_neighborhoods['Neighborhood'].astype(str) + '-Toronto'
toronto_neighborhoods.drop(columns=['PostalCode'], inplace =True)
toronto_neighborhoods.head()

,Unnamed: 0,Borough,Neighborhood,Latitude,Longitude
0,4,Downtown Toronto,Regent Park / Harbourfront-Toronto,43.654260,-79.360636
1,6,Downtown Toronto,Queen's Park / Ontario Provincial Government-T...,43.662301,-79.389494
2,13,Downtown Toronto,"Garden District, Ryerson-Toronto",43.657162,-79.378937
3,22,Downtown Toronto,St. James Town-Toronto,43.651494,-79.375418
4,30,East Toronto,The Beaches-Toronto,43.676357,-79.293031


In [0]:
toronto_neighborhoods.to_csv(r"/content/drive/My Drive/Colab Notebooks/Data/PROJ/2019_10_18_pd_sk_fq_tb/data/toronto_neighborhoods.csv", sep = ',')

In [0]:
toronto_neighborhoods = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Data/PROJ/2019_10_18_pd_sk_fq_tb/data/toronto_neighborhoods.csv", sep = ',', low_memory=False)

In [0]:
toronto_boroughs = (toronto_neighborhoods['Borough'])
toronto_boroughs.unique()

array(['Downtown Toronto', 'East Toronto', 'West Toronto',
       'Central Toronto'], dtype=object)

Check the number of boroughs and neighborhoods in Toronto

In [0]:
print('The Toronto dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_neighborhoods['Borough'].unique()),
        toronto_neighborhoods.shape[0]
    )
)

The Toronto dataframe has 4 boroughs and 39 neighborhoods.


_________________________________________________________________


#### Use geopy library to get the latitude and longitude values of New York City.

In [0]:
address = 'New York City, NY'
import warnings
warnings.filterwarnings("ignore")
geolocator = Nominatim()
newyork_location = geolocator.geocode(address)
newyork_latitude = newyork_location.latitude
newyork_longitude = newyork_location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(newyork_latitude, newyork_longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


#### Use geopy library to get the latitude and longitude values of Toronto.

In [0]:
address = 'Toronto, Ontario'
import warnings
warnings.filterwarnings("ignore")
geolocator = Nominatim()
toronto_location = geolocator.geocode(address)
toronto_latitude = toronto_location.latitude
toronto_longitude = toronto_location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(toronto_latitude, toronto_longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Define Foursquare Credentials and Version

In [0]:
CLIENT_ID = '0MJA3NYYG3U2ZY1LTZN2OYEHS3Y3WVSON2GBSO3IL4EDYVIR' 
CLIENT_SECRET = 'WGWSAF2TKVUQPE3PD0N3EOITFVBY5EYP1VCZI3BMUG0ROUS5' 
VERSION = '20180605' 

#### Combine both cities' neighborhood.

In [0]:
neighborhoods = pd.concat([newyork_neighborhoods, toronto_neighborhoods])
neighborhoods_data = neighborhoods.reset_index(drop=True)
neighborhoods_data.shape

(345, 6)

In [0]:
neighborhoods_data.head()

,Borough,Neighborhood,Latitude,Longitude,Unnamed: 0,Unnamed: 0.1
0,Bronx,"Wakefield, Bronx",40.894705,-73.847201,NaN,NaN
1,Bronx,"Co-op City, Bronx",40.874294,-73.829939,NaN,NaN
2,Bronx,"Eastchester, Bronx",40.887556,-73.827806,NaN,NaN
3,Bronx,"Fieldston, Bronx",40.895437,-73.905643,NaN,NaN
4,Bronx,"Riverdale, Bronx",40.890834,-73.912585,NaN,NaN


In [0]:
print('The combined neighborhoods dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods_data['Borough'].unique()),
        neighborhoods_data.shape[0]
    )
)

The combined neighborhoods dataframe has 9 boroughs and 345 neighborhoods.


In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT = 100 # limit of number of venues returned by Foursquare API

    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [0]:
neighborhoods_venues = getNearbyVenues(names=neighborhoods_data['Neighborhood'],
                                   latitudes=neighborhoods_data['Latitude'],
                                   longitudes=neighborhoods_data['Longitude']
                                  )

Wakefield, Bronx
Co-op City, Bronx
Eastchester, Bronx
Fieldston, Bronx
Riverdale, Bronx
Kingsbridge, Bronx
Marble Hill, Manhattan
Woodlawn, Bronx
Norwood, Bronx
Williamsbridge, Bronx
Baychester, Bronx
Pelham Parkway, Bronx
City Island, Bronx
Bedford Park, Bronx
University Heights, Bronx
Morris Heights, Bronx
Fordham, Bronx
East Tremont, Bronx
West Farms, Bronx
High  Bridge, Bronx
Melrose, Bronx
Mott Haven, Bronx
Port Morris, Bronx
Longwood, Bronx
Hunts Point, Bronx
Morrisania, Bronx
Soundview, Bronx
Clason Point, Bronx
Throgs Neck, Bronx
Country Club, Bronx
Parkchester, Bronx
Westchester Square, Bronx
Van Nest, Bronx
Morris Park, Bronx
Belmont, Bronx
Spuyten Duyvil, Bronx
North Riverdale, Bronx
Pelham Bay, Bronx
Schuylerville, Bronx
Edgewater Park, Bronx
Castle Hill, Bronx
Olinville, Bronx
Pelham Gardens, Bronx
Concourse, Bronx
Unionport, Bronx
Edenwald, Bronx
Bay Ridge, Brooklyn
Bensonhurst, Brooklyn
Sunset Park, Brooklyn
Greenpoint, Brooklyn
Gravesend, Brooklyn
Brighton Beach, Brookl

#### Let's check the size of the resulting dataframe

In [0]:
print(neighborhoods_venues.shape)
neighborhoods_venues.head()

(11924, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Wakefield, Bronx",40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,"Wakefield, Bronx",40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
2,"Wakefield, Bronx",40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
3,"Wakefield, Bronx",40.894705,-73.847201,Walgreens,40.896528,-73.844700,Pharmacy
4,"Wakefield, Bronx",40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop


In [0]:
neighborhoods_venues.to_csv(r"/content/drive/My Drive/Colab Notebooks/Data/PROJ/2019_10_18_pd_sk_fq_tb/data/neighborhoods_venues.csv", sep = ',')

Let's check how many venues were returned for each neighborhood

In [0]:
neighborhoods_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Allerton, Bronx",31,31,31,31,31,31
"Annadale, Staten Island",9,9,9,9,9,9
"Arden Heights, Staten Island",6,6,6,6,6,6
"Arlington, Staten Island",7,7,7,7,7,7
"Arrochar, Staten Island",21,21,21,21,21,21
"Arverne, Queens",18,18,18,18,18,18
"Astoria Heights, Queens",11,11,11,11,11,11
"Astoria, Queens",100,100,100,100,100,100
"Auburndale, Queens",19,19,19,19,19,19


#### Let's find out how many unique categories can be curated from all the returned venues

In [0]:
print('There are {} uniques categories.'.format(len(neighborhoods_venues['Venue Category'].unique())))

There are 454 uniques categories.


## Methodology

From the Foursquare API's results, we can find the category of venues. The unique categories for all combined neighborhoods are:

In [0]:
unique = neighborhoods_venues['Venue Category'].unique()
venue_category_unique =pd.DataFrame(unique, columns =['Venue Categories'])

In [0]:
venue_category_unique.head()

,Venue Categories
0,Dessert Shop
1,Pharmacy
2,Ice Cream Shop
3,Donut Shop
4,Gas Station


In [0]:
venue_category_unique.shape

(454, 1)

In [0]:
venue_category_unique.to_csv(r"/content/drive/My Drive/Colab Notebooks/Data/PROJ/2019_10_18_pd_sk_fq_tb/data/venue_category_unique.csv", sep = ',')

We set the dummy variables for each venue's category for each neighborhood.

In [0]:
# one hot encoding
neighborhoods_onehot = pd.get_dummies(neighborhoods_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
neighborhoods_onehot['Neighborhood'] = neighborhoods_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [neighborhoods_onehot.columns[-1]] + list(neighborhoods_onehot.columns[:-1])
neighborhoods_onehot = neighborhoods_onehot[fixed_columns]

neighborhoods_onehot.head()

,Yoga Studio,Accessories Store,Acupuncturist,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Auditorium,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Rec Center,College Stadium,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Distribution Center,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Knitting Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Medical School,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [0]:
neighborhoods_grouped = neighborhoods_onehot.groupby('Neighborhood').mean().reset_index()
neighborhoods_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Acupuncturist,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Auditorium,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Rec Center,College Stadium,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Distribution Center,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Knitting Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Medical School,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Easte

There are 344 neighborhoods with 444 categories.

In [0]:
neighborhoods_grouped.shape

(344, 454)

_________________________________________________________________


## Finally, let's find out the 3 most common venue categories for each neighborhood.

In [0]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

A new dataframe is created to display the 3 most common venue categories for each neighborhood.

In [0]:
num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = neighborhoods_grouped['Neighborhood']

for ind in np.arange(neighborhoods_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(neighborhoods_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,"Allerton, Bronx",Pizza Place,Department Store,Deli / Bodega
1,"Annadale, Staten Island",Pizza Place,Pub,Diner
2,"Arden Heights, Staten Island",Moving Target,Pool,Pharmacy
3,"Arlington, Staten Island",Bus Stop,Home Service,Intersection
4,"Arrochar, Staten Island",Bus Stop,Pizza Place,Bagel Shop


The Top 3 Venue Categories is ranked among
the 3 most common venues for each neighborhood.

In [0]:
neighborhoods_venues_first = neighborhoods_venues_sorted[['Neighborhood', '1st Most Common Venue']]
neighborhoods_venues_first.head()

,Neighborhood,1st Most Common Venue
0,"Allerton, Bronx",Pizza Place
1,"Annadale, Staten Island",Pizza Place
2,"Arden Heights, Staten Island",Moving Target
3,"Arlington, Staten Island",Bus Stop
4,"Arrochar, Staten Island",Bus Stop


In [0]:
neighborhoods_venues_first.groupby('1st Most Common Venue').count().sort_values(by=['Neighborhood'], ascending=False).head(3)

,Neighborhood
1st Most Common Venue,
Pizza Place,33
Coffee Shop,29
Italian Restaurant,22


In [0]:
neighborhoods_ranked = neighborhoods_grouped[['Neighborhood', 'Pizza Place','Coffee Shop','Italian Restaurant']]
neighborhoods_ranked.head()

,Neighborhood,Pizza Place,Coffee Shop,Italian Restaurant
0,"Allerton, Bronx",0.193548,0.000000,0.000000
1,"Annadale, Staten Island",0.222222,0.000000,0.000000
2,"Arden Heights, Staten Island",0.166667,0.166667,0.000000
3,"Arlington, Staten Island",0.000000,0.142857,0.000000
4,"Arrochar, Staten Island",0.095238,0.000000,0.095238


In [0]:
len(neighborhoods_ranked)

344

In [0]:
neighborhoods_merged = neighborhoods_data.sort_values('Neighborhood').drop('Borough', 1)
neighborhoods_merged = neighborhoods_merged.join(neighborhoods_ranked.set_index('Neighborhood'), on='Neighborhood')
neighborhoods_merged.head()

,Neighborhood,Latitude,Longitude,Unnamed: 0,Unnamed: 0.1,Pizza Place,Coffee Shop,Italian Restaurant
298,"Allerton, Bronx",40.865788,-73.859319,NaN,NaN,0.193548,0.000000,0.000000
215,"Annadale, Staten Island",40.538114,-74.178549,NaN,NaN,0.222222,0.000000,0.000000
241,"Arden Heights, Staten Island",40.549286,-74.185887,NaN,NaN,0.166667,0.166667,0.000000
227,"Arlington, Staten Island",40.635325,-74.165104,NaN,NaN,0.000000,0.142857,0.000000
228,"Arrochar, Staten Island",40.596313,-74.067124,NaN,NaN,0.095238,0.000000,0.095238


In [0]:
neighborhoods_merged.to_csv(r"/content/drive/My Drive/Colab Notebooks/Data/PROJ/2019_10_18_pd_sk_fq_tb/data/neighborhoods_merged.csv", sep = ',')

In [0]:
neighborhoods_merged = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Data/PROJ/2019_10_18_pd_sk_fq_tb/data/neighborhoods_merged.csv", sep = ',', low_memory=False)

##  > 3 most common venue categories selected in New York

In [0]:
neighborhoods_ny_merged = neighborhoods_merged[neighborhoods_merged['Neighborhood'].str.endswith('-Toronto') == False]
neighborhoods_ny_merged.head()

,Unnamed: 0,Neighborhood,Latitude,Longitude,Unnamed: 0.1,Unnamed: 0.1.1,Pizza Place,Coffee Shop,Italian Restaurant
0,298,"Allerton, Bronx",40.865788,-73.859319,NaN,NaN,0.193548,0.000000,0.000000
1,215,"Annadale, Staten Island",40.538114,-74.178549,NaN,NaN,0.222222,0.000000,0.000000
2,241,"Arden Heights, Staten Island",40.549286,-74.185887,NaN,NaN,0.166667,0.166667,0.000000
3,227,"Arlington, Staten Island",40.635325,-74.165104,NaN,NaN,0.000000,0.142857,0.000000
4,228,"Arrochar, Staten Island",40.596313,-74.067124,NaN,NaN,0.095238,0.000000,0.095238


In [0]:
len(neighborhoods_ny_merged)

306

In [0]:
neighborhoods_ny_pizza = neighborhoods_ny_merged[['Neighborhood','Latitude','Longitude','Pizza Place']]
neighborhoods_ny_pizza = neighborhoods_ny_pizza[neighborhoods_ny_pizza['Pizza Place'] > 0]
neighborhoods_ny_pizza.head()

,Neighborhood,Latitude,Longitude,Pizza Place
0,"Allerton, Bronx",40.865788,-73.859319,0.193548
1,"Annadale, Staten Island",40.538114,-74.178549,0.222222
2,"Arden Heights, Staten Island",40.549286,-74.185887,0.166667
4,"Arrochar, Staten Island",40.596313,-74.067124,0.095238
5,"Arverne, Queens",40.589144,-73.791992,0.055556


In [0]:
len(neighborhoods_ny_pizza)

204

In [0]:
search_values = ['Pizza Place', 'Coffee Shop', 'Italian Restaurant']

In [0]:
neighborhoods_all_venue_names = neighborhoods_venues[neighborhoods_venues['Venue Category'].str.contains('|'.join(search_values))]

In [0]:
neighborhoods_all_venue_names.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
11,"Co-op City, Bronx",40.874294,-73.829939,Capri II Pizza,40.876374,-73.829940,Pizza Place
27,"Eastchester, Bronx",40.887556,-73.827806,Mario's Pizza,40.888628,-73.831260,Pizza Place
63,"Kingsbridge, Bronx",40.881687,-73.902818,Kingsbridge Social Club,40.884545,-73.901964,Pizza Place
65,"Kingsbridge, Bronx",40.881687,-73.902818,Sam's Pizza,40.879435,-73.905859,Pizza Place
73,"Kingsbridge, Bronx",40.881687,-73.902818,Mon Amour Coffee & Wine,40.885009,-73.900332,Coffee Shop


In [0]:
neighborhoods_all_venue_names.shape

(1305, 7)

In [0]:
neighborhoods_all_venue_names.to_csv(r"/content/drive/My Drive/Colab Notebooks/Data/PROJ/2019_10_18_pd_sk_fq_tb/data/neighborhoods_all_venue_names.csv", sep = ',')

In [0]:
neighborhoods_all_venue_pizza = neighborhoods_all_venue_names[neighborhoods_all_venue_names['Venue Category'].str.endswith('Pizza Place') == True]

In [0]:
neighborhoods_all_venue_pizza.shape

(482, 7)

In [0]:
neighborhoods_all_venue_pizza.to_csv(r"/content/drive/My Drive/Colab Notebooks/Data/PROJ/2019_10_18_pd_sk_fq_tb/data/neighborhoods_all_venue_pizza.csv", sep = ',')

In [0]:
neighborhoods_all_venue_coffee = neighborhoods_all_venue_names[neighborhoods_all_venue_names['Venue Category'].str.startswith('Coffee Shop') == True]

In [0]:
neighborhoods_all_venue_coffee.shape

(470, 7)

In [0]:
neighborhoods_all_venue_coffee.to_csv(r"/content/drive/My Drive/Colab Notebooks/Data/PROJ/2019_10_18_pd_sk_fq_tb/data/neighborhoods_all_venue_coffee.csv", sep = ',')

In [0]:
neighborhoods_all_venue_italian = neighborhoods_all_venue_names[neighborhoods_all_venue_names['Venue Category'].str.startswith('Italian') == True]

In [0]:
neighborhoods_all_venue_italian.shape

(353, 7)

In [0]:
neighborhoods_all_venue_italian.to_csv(r"/content/drive/My Drive/Colab Notebooks/Data/PROJ/2019_10_18_pd_sk_fq_tb/data/neighborhoods_all_venue_italian.csv", sep = ',')

In [0]:
neighborhoods_ny_venue_names = neighborhoods_all_venue_names[neighborhoods_all_venue_names['Neighborhood'].str.endswith('-Toronto') == False]

In [0]:
neighborhoods_ny_venue_names.shape

(1076, 7)

In [0]:
neighborhoods_ny_venue_names.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
11,"Co-op City, Bronx",40.874294,-73.829939,Capri II Pizza,40.876374,-73.829940,Pizza Place
27,"Eastchester, Bronx",40.887556,-73.827806,Mario's Pizza,40.888628,-73.831260,Pizza Place
63,"Kingsbridge, Bronx",40.881687,-73.902818,Kingsbridge Social Club,40.884545,-73.901964,Pizza Place
65,"Kingsbridge, Bronx",40.881687,-73.902818,Sam's Pizza,40.879435,-73.905859,Pizza Place
73,"Kingsbridge, Bronx",40.881687,-73.902818,Mon Amour Coffee & Wine,40.885009,-73.900332,Coffee Shop


In [0]:
neighborhoods_ny_venue_names_boroughs = pd.merge(newyork_neighborhoods[['Borough','Neighborhood']], neighborhoods_ny_venue_names[['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']], how ='inner', on ='Neighborhood')

In [0]:
neighborhoods_ny_venue_names_boroughs.head()

,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bronx,"Co-op City, Bronx",40.874294,-73.829939,Capri II Pizza,40.876374,-73.829940,Pizza Place
1,Bronx,"Eastchester, Bronx",40.887556,-73.827806,Mario's Pizza,40.888628,-73.831260,Pizza Place
2,Bronx,"Kingsbridge, Bronx",40.881687,-73.902818,Kingsbridge Social Club,40.884545,-73.901964,Pizza Place
3,Bronx,"Kingsbridge, Bronx",40.881687,-73.902818,Sam's Pizza,40.879435,-73.905859,Pizza Place
4,Bronx,"Kingsbridge, Bronx",40.881687,-73.902818,Mon Amour Coffee & Wine,40.885009,-73.900332,Coffee Shop


In [0]:
neighborhoods_ny_venue_names_boroughs.shape

(1076, 8)

In [0]:
neighborhoods_ny_venue_names_boroughs.to_csv(r"/content/drive/My Drive/Colab Notebooks/Data/PROJ/2019_10_18_pd_sk_fq_tb/data/neighborhoods_ny_venue_names_boroughs.csv", sep = ',')

In [0]:
neighborhoods_ny_coffee = neighborhoods_ny_merged[['Neighborhood','Latitude','Longitude','Coffee Shop']]
neighborhoods_ny_coffee = neighborhoods_ny_coffee[neighborhoods_ny_coffee['Coffee Shop'] > 0]
neighborhoods_ny_coffee.head()

,Neighborhood,Latitude,Longitude,Coffee Shop
2,"Arden Heights, Staten Island",40.549286,-74.185887,0.166667
3,"Arlington, Staten Island",40.635325,-74.165104,0.142857
5,"Arverne, Queens",40.589144,-73.791992,0.055556
7,"Astoria, Queens",40.768509,-73.915654,0.020000
9,"Bath Beach, Brooklyn",40.599519,-73.998752,0.022222


In [0]:
len(neighborhoods_ny_coffee)

122

In [0]:
neighborhoods_ny_italian = neighborhoods_ny_merged[['Neighborhood','Latitude','Longitude','Italian Restaurant']]
neighborhoods_ny_italian = neighborhoods_ny_italian[neighborhoods_ny_italian['Italian Restaurant'] > 0]
neighborhoods_ny_italian.head()

,Neighborhood,Latitude,Longitude,Italian Restaurant
4,"Arrochar, Staten Island",40.596313,-74.067124,0.095238
6,"Astoria Heights, Queens",40.770317,-73.894680,0.090909
7,"Astoria, Queens",40.768509,-73.915654,0.020000
8,"Auburndale, Queens",40.761730,-73.791762,0.105263
9,"Bath Beach, Brooklyn",40.599519,-73.998752,0.044444


In [0]:
len(neighborhoods_ny_italian)

121

## > 3 most common venue categories selected in Toronto

In [0]:
neighborhoods_to_merged = neighborhoods_merged[neighborhoods_merged['Neighborhood'].str.endswith('-Toronto') == True]
neighborhoods_to_merged.head()

,Unnamed: 0,Neighborhood,Latitude,Longitude,Unnamed: 0.1,Unnamed: 0.1.1,Pizza Place,Coffee Shop,Italian Restaurant
25,311,Berczy Park-Toronto,43.644771,-79.373306,5.0,31.0,0.0000,0.090909,0.000000
36,320,Brockton / Parkdale Village / Exhibition Place...,43.636847,-79.428191,14.0,68.0,0.0000,0.090909,0.045455
43,344,Business reply mail Processing CentrE-Toronto,43.662744,-79.321558,38.0,168.0,0.0625,0.000000,0.000000
45,338,CN Tower / King and Spadina / Railway Lands / ...,43.628947,-79.394420,32.0,139.0,0.0000,0.062500,0.000000
52,312,Central Bay Street-Toronto,43.657952,-79.387383,6.0,40.0,0.0000,0.181818,0.051948


In [0]:
len(neighborhoods_to_merged)

39

In [0]:
search_values_to = ['Pizza Place', 'Coffee Shop', 'Italian Restaurant']

In [0]:
neighborhoods_to_venue_names = neighborhoods_venues[neighborhoods_venues['Venue Category'].str.contains('|'.join(search_values))]

In [0]:
neighborhoods_to_venue_names.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
11,"Co-op City, Bronx",40.874294,-73.829939,Capri II Pizza,40.876374,-73.829940,Pizza Place
27,"Eastchester, Bronx",40.887556,-73.827806,Mario's Pizza,40.888628,-73.831260,Pizza Place
63,"Kingsbridge, Bronx",40.881687,-73.902818,Kingsbridge Social Club,40.884545,-73.901964,Pizza Place
65,"Kingsbridge, Bronx",40.881687,-73.902818,Sam's Pizza,40.879435,-73.905859,Pizza Place
73,"Kingsbridge, Bronx",40.881687,-73.902818,Mon Amour Coffee & Wine,40.885009,-73.900332,Coffee Shop


In [0]:
neighborhoods_to_venue_names = neighborhoods_to_venue_names[neighborhoods_to_venue_names['Neighborhood'].str.endswith('-Toronto') == True]

In [0]:
neighborhoods_to_venue_names.shape

(229, 7)

In [0]:
neighborhoods_to_venue_names.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
10244,Regent Park / Harbourfront-Toronto,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
10255,Regent Park / Harbourfront-Toronto,43.65426,-79.360636,Rooster Coffee,43.651900,-79.365609,Coffee Shop
10257,Regent Park / Harbourfront-Toronto,43.65426,-79.360636,Arvo,43.649963,-79.361442,Coffee Shop
10258,Regent Park / Harbourfront-Toronto,43.65426,-79.360636,Sumach Espresso,43.658135,-79.359515,Coffee Shop
10260,Regent Park / Harbourfront-Toronto,43.65426,-79.360636,Starbucks,43.651613,-79.364917,Coffee Shop


In [0]:
neighborhoods_to_venue_names_boroughs = pd.merge(toronto_neighborhoods[['Borough','Neighborhood']], neighborhoods_to_venue_names[['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']], how ='inner', on ='Neighborhood')

In [0]:
neighborhoods_to_venue_names_boroughs.head()

,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Downtown Toronto,Regent Park / Harbourfront-Toronto,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,Downtown Toronto,Regent Park / Harbourfront-Toronto,43.65426,-79.360636,Rooster Coffee,43.651900,-79.365609,Coffee Shop
2,Downtown Toronto,Regent Park / Harbourfront-Toronto,43.65426,-79.360636,Arvo,43.649963,-79.361442,Coffee Shop
3,Downtown Toronto,Regent Park / Harbourfront-Toronto,43.65426,-79.360636,Sumach Espresso,43.658135,-79.359515,Coffee Shop
4,Downtown Toronto,Regent Park / Harbourfront-Toronto,43.65426,-79.360636,Starbucks,43.651613,-79.364917,Coffee Shop


In [0]:
neighborhoods_to_venue_names_boroughs.shape

(229, 8)

In [0]:
neighborhoods_to_venue_names_boroughs.to_csv(r"/content/drive/My Drive/Colab Notebooks/Data/PROJ/2019_10_18_pd_sk_fq_tb/data/neighborhoods_to_venue_names_boroughs.csv", sep = ',')

In [0]:
neighborhoods_to_pizza = neighborhoods_to_merged[['Neighborhood','Latitude','Longitude','Pizza Place']]
neighborhoods_to_pizza = neighborhoods_to_pizza[neighborhoods_to_pizza['Pizza Place'] > 0]
neighborhoods_to_pizza.head()

,Neighborhood,Latitude,Longitude,Pizza Place
43,Business reply mail Processing CentrE-Toronto,43.662744,-79.321558,0.062500
59,Church and Wellesley-Toronto,43.665860,-79.383160,0.012658
71,Commerce Court / Victoria Hotel-Toronto,43.648198,-79.379817,0.010000
81,Davisville-Toronto,43.704324,-79.388790,0.083333
109,First Canadian Place / Underground city-Toronto,43.648429,-79.382280,0.020000


In [0]:
len(neighborhoods_to_pizza)

18

In [0]:
neighborhoods_to_coffee = neighborhoods_to_merged[['Neighborhood','Latitude','Longitude','Coffee Shop']]
neighborhoods_to_coffee = neighborhoods_to_coffee[neighborhoods_to_coffee['Coffee Shop'] > 0]
neighborhoods_to_coffee.head()

,Neighborhood,Latitude,Longitude,Coffee Shop
25,Berczy Park-Toronto,43.644771,-79.373306,0.090909
36,Brockton / Parkdale Village / Exhibition Place...,43.636847,-79.428191,0.090909
45,CN Tower / King and Spadina / Railway Lands / ...,43.628947,-79.394420,0.062500
52,Central Bay Street-Toronto,43.657952,-79.387383,0.181818
58,Christie-Toronto,43.669542,-79.422564,0.105263


In [0]:
len(neighborhoods_to_coffee)

30

In [0]:
neighborhoods_to_italian = neighborhoods_to_merged[['Neighborhood','Latitude','Longitude','Italian Restaurant']]
neighborhoods_to_italian = neighborhoods_to_italian[neighborhoods_to_italian['Italian Restaurant'] > 0]
neighborhoods_to_italian.head()

,Neighborhood,Latitude,Longitude,Italian Restaurant
36,Brockton / Parkdale Village / Exhibition Place...,43.636847,-79.428191,0.045455
52,Central Bay Street-Toronto,43.657952,-79.387383,0.051948
58,Christie-Toronto,43.669542,-79.422564,0.052632
59,Church and Wellesley-Toronto,43.665860,-79.383160,0.012658
71,Commerce Court / Victoria Hotel-Toronto,43.648198,-79.379817,0.020000


In [0]:
len(neighborhoods_to_italian)

21

## Results

The 3 most common venue categories selected in New York City.

In [0]:
# create map of New York
map_newyork = folium.Map(location=[newyork_latitude, newyork_longitude], zoom_start=11)

# add Pizza Places to the map
markers_colors = []
for lat, lon, poi in zip(neighborhoods_ny_pizza['Latitude'], neighborhoods_ny_pizza['Longitude'], neighborhoods_ny_pizza['Neighborhood']):
    label = folium.Popup(str(poi), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        color='RED',
        fill_opacity=1.0).add_to(map_newyork)

# add Coffee Shops to the map
markers_colors = []
for lat, lon, poi in zip(neighborhoods_ny_coffee['Latitude'], neighborhoods_ny_coffee['Longitude'], neighborhoods_ny_coffee['Neighborhood']):
    label = folium.Popup(str(poi), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        color='MAROON',
        fill_opacity=1.0).add_to(map_newyork)

# add Italian Restaurants to the map
markers_colors = []
for lat, lon, poi in zip(neighborhoods_ny_italian['Latitude'], neighborhoods_ny_italian['Longitude'], neighborhoods_ny_italian['Neighborhood']):
    label = folium.Popup(str(poi), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        color='GREEN',
        fill_opacity=1.0).add_to(map_newyork)
       
map_newyork

The 3 most common venue categories selected in Toronto.

In [0]:
# create map of Toronto
map_toronto = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=11)

# add Pizza Places to the map
markers_colors = []
for lat, lon, poi in zip(neighborhoods_to_pizza['Latitude'], neighborhoods_to_pizza['Longitude'], neighborhoods_to_pizza['Neighborhood']):
    label = folium.Popup(str(poi), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        color='RED',
        fill_opacity=1.0).add_to(map_toronto)

# add Coffee Shops to the map
markers_colors = []
for lat, lon, poi in zip(neighborhoods_to_coffee['Latitude'], neighborhoods_to_coffee['Longitude'], neighborhoods_to_coffee['Neighborhood']):
    label = folium.Popup(str(poi), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        color='MAROON',
        fill_opacity=1.0).add_to(map_toronto)

# add Italian Restaurants to the map
markers_colors = []
for lat, lon, poi in zip(neighborhoods_to_italian['Latitude'], neighborhoods_to_italian['Longitude'], neighborhoods_to_italian['Neighborhood']):
    label = folium.Popup(str(poi), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        color='GREEN',
        fill_opacity=1.0).add_to(map_toronto)
       
map_toronto

## Insight Generated

The 3 most common venue categories in Toronto and New York City combined are:

Pizza Places	
Coffee Shops	
Italian Restaurants	